In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [24]:
input_folder = '../data/railways/global/' 
input_file = input_folder+'Africa_main.csv'

output_folder = '../data/output_images/Africa/'
output_csv = output_folder+'AFR.csv'

### Import Routes

In [9]:
routes = pd.read_csv(input_file)

routes.head()

,Way ID (track segment),Node ID,Latitude,Longitude,Elevation(feet)
0,4064202,25326744,-33.960374,25.461479,600.4
1,4064202,603658924,-33.962616,25.466112,587.3
2,4064202,603658955,-33.962988,25.466851,584.0
3,4064202,603658957,-33.963161,25.467242,584.0
4,4064202,25326745,-33.963285,25.467578,584.0


In [4]:
# Number of routes
routes.shape

(907143, 5)

In [5]:
'''
Get all points
'''

# len(routes['Node ID'].tolist())

points = []
for i in range(0,10000):
    try:
        route = routes.iloc[i]
        sec_points = (route['Longitude'],route['Latitude'])
        points.append(sec_points)
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

10000 points


In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [6]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),200):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

50


In [7]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,25.461478699999997_-33.9603741,25.461479,-33.960374
1,28.354649_-25.7229153,28.354649,-25.722915
2,25.54801_-33.984798100000006,25.548010,-33.984798
3,28.071962300000003_-25.5532706,28.071962,-25.553271
4,18.4334862_-34.135386,18.433486,-34.135386


In [13]:
'''
Export points
'''

df.to_csv(file_out,header=None)

In [ ]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [ ]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

In [ ]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+output_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
# url = 'https://www.google.com/maps/@?api=1&map_action=map'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 19,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 